In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import time
import cPickle
import os
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

In [2]:
def getFrameData(root, iterKey):
    allFrameData = []
    for frameData in root.iter(iterKey):
        if not frameData.attrib:
            allFrameData.append(frameData.text)
        else:
            allFrameData.append(frameData.attrib)
    return allFrameData

def readXMLFile(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()
    parent = {}
    for child in root:
        tempList = list(child)
    #     print child.tag
        if not tempList:
            if not child.attrib:
                parent[child.tag] = child.text
            else:
                parent[child.tag] = child.attrib
        else:
            parent[child.tag] = {}
            for grand_child in child:
    #             print '\t', grand_child.tag
                tempList = list(grand_child)
                if not tempList:
                    if not grand_child.attrib:
                        parent[child.tag][grand_child.tag] = grand_child.text
                    else:
                        parent[child.tag][grand_child.tag] = grand_child.attrib
                else:
                    parent[child.tag][grand_child.tag] = {}
                    for grand_grand_child in grand_child:
    #                     print '\t\t', grand_grand_child.tag
                        tempList = list(grand_grand_child)
                        if not tempList:
                            if not grand_grand_child.attrib:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.text
                            else:
                                parent[child.tag][grand_child.tag][grand_grand_child.tag] = grand_grand_child.attrib
                        else:
                            parent[child.tag][grand_child.tag][grand_grand_child.tag] = {}
                            for grand_grand_grand_child in grand_grand_child:
    #                             print '\t\t\t', grand_grand_grand_child.tag
                                tempList = list(grand_grand_grand_child)
                                if not tempList:
                                    if not grand_grand_grand_child.attrib:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                    else:
                                        parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                else:
                                    parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag] = {}
                                    for grand_grand_grand_grand_child in grand_grand_grand_child:
            #                             print '\t\t\t', grand_grand_grand_grand_child.tag
                                        tempList = list(grand_grand_grand_grand_child)
                                        if not tempList:
                                            if not grand_grand_grand_grand_child.attrib:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.text
                                            else:
                                                parent[child.tag][grand_child.tag][grand_grand_child.tag][grand_grand_grand_child.tag][grand_grand_grand_child.tag] = grand_grand_grand_child.attrib
                                        else:
                                            print "There are more grand childrens than you think"

    parent['kinetics'] = []
    parent['position'] = {}
    parent['ruleData'] = []
    parent['stimulus'] = []

    larvaFrameData = pd.DataFrame(getFrameData(root, 'larvaFrameData'))
    skeleton = pd.DataFrame(getFrameData(root, 'skeleton'))
    parent['kinetics'] = pd.concat([larvaFrameData, skeleton])
    parent['position']['midpoint'] = pd.DataFrame(getFrameData(root, 'midpoint'))
    parent['position']['centroid'] = pd.DataFrame(getFrameData(root, 'centroid'))
    parent['position']['head'] = pd.DataFrame(getFrameData(root, 'head'))
    parent['position']['tail'] = pd.DataFrame(getFrameData(root, 'tail'))
    parent['position']['featureLocation'] = pd.DataFrame(getFrameData(root, 'featureLocation'))
    parent['ruleData'] = pd.DataFrame(getFrameData(root, 'ruleData'))

    stimulusIntensity = pd.DataFrame(getFrameData(root, 'intensityPercentage'))
    temp = pd.DataFrame(getFrameData(root, 'ledArrayStimulus'))
    if not temp.empty:
        parent['stimulus'] = pd.concat([temp, stimulusIntensity], axis=1)
        parent['stimulus'].rename(columns={0: 'intensity'}, inplace=True)
    else:
        parent['stimulus'] = stimulusIntensity

    return parent

In [5]:
# data_dir = '20180316_OK6_CsChrimson_Tail_200micron'
data_dir = '20180316_OK6_CsChrimson_Tail_200micron'

all_xml_files = []
root_path = '/Volumes/GoogleDrive/My Drive/CRG_Dropbox/AljoComputer/Matlab/high_res_tracker/Data/'
data_dir = os.path.join(root_path,data_dir,'XML')
for file in os.listdir(data_dir):
    if file.endswith(".xml"):
        print file
        all_xml_files.append(os.path.join(data_dir, file))

venkman-log-20180316-062750100-sid-1.xml
venkman-log-20180316-063825616-sid-2.xml
venkman-log-20180316-065932551-sid-3.xml
venkman-log-20180316-070653899-sid-4.xml
venkman-log-20180316-072006474-sid-5.xml
venkman-log-20180316-072842978-sid-6.xml
venkman-log-20180316-073732321-sid-7.xml
venkman-log-20180316-080508653-sid-8.xml
venkman-log-20180316-081146603-sid-9.xml
venkman-log-20180316-083055080-sid-0.xml
venkman-log-20180316-083802623-sid-1.xml
venkman-log-20180316-084618499-sid-2.xml
venkman-log-20180316-085206920-sid-3.xml
venkman-log-20180316-090115098-sid-4.xml
venkman-log-20180316-091040680-sid-5.xml


In [ ]:
## Parse Files in Parallel
tStart = time.time()
kin_data = []
kin_data = Parallel(n_jobs=num_cores)(delayed(readXMLFile)(xml_file) for xml_file in all_xml_files)
print 'Time taken to parse %d files in parallel is %0.3f sec'%(len(all_xml_files), time.time()-tStart)

Time taken to parse 15 files in parallel is 634.835 sec


In [ ]:
## Save Parsed Files
cPickle.dump(kin_data, open(os.path.join(data_dir, 'kinData_xml.pkl'), 'wb'), protocol=2)

In [ ]:
## Parse Files Sequentially
# c=0
# for xml_file in all_xml_files:
#     tStart = time.time()
#     temp = readXMLFile(xml_file)
#     kin_data.append(readXMLFile(xml_file))
#     c+=1
#     print 'Time taken to parse file %d is %0.3f'%(c, time.time()-tStart)